In [ ]:
import csv
import json
import requests
import numpy as np
import pandas as pd
import time
import datetime


df_game = pd.read_csv('C:/Users/cwh10/sampleMatchData.csv',index_col=0, encoding='utf-8')
print(df_game)

df_champion = pd.read_csv('champion_info.csv')
df_champion.tail()

champion_id_list = list(df_champion.Key)
champion_id_list.sort()


In [ ]:
import pandas as pd
import ast
def champion_win_rate(champion_id,graph):
    win,count = graph[champion_id]
    winrate = (win/count)*100
    return [winrate, win, count]

df_game = pd.read_csv('C:/Users/cwh10/sampleMatchData.csv')

# for i in df_game.loc[:,'participants']:
#     print(i['championId'])

dict = {}
for i in range(len(df_game)):
    row = df_game.iloc[i] # 한 줄씩
    # 여기서 부터 participant에서 이용
    participants=ast.literal_eval(row[12]) #10명의 플레이어 list 1판기준임
    for j in participants:
        champ_id =j['championId']
        if champ_id not in dict:
            if j['stats']['win'] == True:
                dict[champ_id]=[1,1] #  이긴수/판수
            else:
                dict[champ_id]=[0,1]
        else:
            if j['stats']['win'] == True:
                dict[champ_id][0] +=1
                dict[champ_id][1] +=1 #  이긴수/판수
            else:
                dict[champ_id][1] +=1 #판수만 추가

print(champion_win_rate(497,dict))

print(champion_id_list)

In [ ]:
champion_win_rates = pd.DataFrame(columns = ['champion_id', 'champion_win_rate', 'game_count', 'champion_win_count'])
for champ_id in champion_id_list:
    if champ_id not in dict:
        continue;
    champion_win_rates = champion_win_rates.append({'champion_id' : champ_id,
                           'champion_win_rate' : champion_win_rate(champ_id,dict)[0],
                           'game_count' : champion_win_rate(champ_id,dict)[1],
                           'champion_win_count' : champion_win_rate(champ_id,dict)[2]},ignore_index=True)

champion_win_rates.reset_index(inplace=True)
del champion_win_rates['index']
print(champion_win_rates)

In [ ]:
def pick_champion_win_rate(champion_id,my_team_champions, opponent_team_champions):
    if (len(my_team_champions)==0)&(len(opponent_team_champions)==0):
        return "Fail"
    my_team_champions.append(champion_id)
    my_team_pick = set(my_team_champions)
    opponent_team_pick = set(opponent_team_champions)
    total_game= 0
    win_game = 0
    df_game = pd.read_csv('C:/Users/cwh10/sampleMatchData.csv')
    for i in range(len(df_game)):
        row = df_game.iloc[i] # 한 줄씩
        # 여기서 부터 participant에서 이용
        participants=ast.literal_eval(row[12]) #10명의 플레이어 list 1판기준임
        team_win= set()
        team_lose =set()
        for j in participants:
            champ_id =j['championId']
            result =j['stats']['win']
            if result == True:
                team_win.add(champ_id)
            else:
                team_lose.add(champ_id)
        if my_team_pick == team_win.intersection(my_team_pick):
            if opponent_team_pick == team_lose.intersection(opponent_team_pick):
                total_game+=1
                win_game+=1
        if opponent_team_pick == team_win.intersection(opponent_team_pick):
            if my_team_pick == team_lose.intersection(my_team_pick):
                total_game+=1
    if total_game == 0:
        return "No Case"
    return [win_game/total_game,total_game,win_game]

print(pick_champion_win_rate(497, [7], [79,126]))     
print(pick_champion_win_rate(497, [], [142]))   
print(pick_champion_win_rate(497, [526], [131,360]))          




# for i in df_game.loc[:,'participants']:
#     print(i['championId'])

# dict = {}
# for i in range(len(df_game)):
#     row = df_game.iloc[i] # 한 줄씩
#     #니가 원하는 기능 여기서 부터 participant에서 이용
#     participants=ast.literal_eval(row[12]) #10명의 플레이어 list 1판기준임
#     for j in participants:
#         champ_id =j['championId']
#         if champ_id not in dict:
#             if j['stats']['win'] == True:
#                 dict[champ_id]=[1,1] #  이긴수/판수
#             else:
#                 dict[champ_id]=[0,1]
#         else:
#             if j['stats']['win'] == True:
#                 dict[champ_id][0] +=1
#                 dict[champ_id][1] +=1 #  이긴수/판수
#             else:
#                 dict[champ_id][1] +=1 #판수만 추가

# print(champion_win_rate(497,dict))
    


In [ ]:
df_champions = pd.read_csv('champion_info.csv')
df_champions.tail()

def championName_to_championId(championName):
    """
    챔피언 이름을 영문으로 넣으면 championId가 나온다.
    :param championName: 챔피언 이름 string
    :return: championId int
    """
    return df_champions.set_index('Champion').to_dict()['Key'][championName]

def championNamelist_to_championIdlist(championNamelist):
    """
    영문으로 된 챔피언 이름 리스트를 넣으면 championId의 리스트가 나온다.
    :param championNamelist: 챔피언 이름 list
    :return: championId list
    """
    return list(pd.Series(championNamelist).apply(lambda x: championName_to_championId(x)))

def recommend_championId(my_team_champions, opponent_team_champions):
    """
    내가 픽 할 상황에서 우리편과 상대편의 픽 된 챔피언 리스트를 넣으면 승률이 가장 높을 것으로 예상되는 상위 5개 챔피언을 보여준다.
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: DataFrame
    """
    df = pd.DataFrame()
    for champion_Id in champion_win_rates[champion_win_rates.game_count >= 300].champion_id:
        df = df.append([[champion_Id, pick_champion_win_rate(champion_Id, my_team_champions, opponent_team_champions)]])
    df.columns = ['id', 'win_rate']
    df.reset_index(inplace=True)
    del df['index']
    df.sort_values('win_rate', ascending=False, inplace=True)
    df_champions = pd.read_csv('champion_info.csv')
    return pd.merge(df, df_champions, on ='id', how='inner').head()